In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
CSV_PATH = 'C:\\Users\\ipl1\\Desktop\\IPL\\세미나(25하계)\\data\\sound\\esc50.csv'
WAV_DIR = 'C:\\Users\\ipl1\\Desktop\\IPL\\세미나(25하계)\\data\\sound\\audio\\audio'

In [3]:
df = pd.read_csv(CSV_PATH)

In [4]:
df.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [5]:
df = df[df["esc10"] == True].reset_index(drop=True)

In [6]:
print(df["category"].unique())

['dog' 'chainsaw' 'crackling_fire' 'helicopter' 'rain' 'crying_baby'
 'clock_tick' 'sneezing' 'rooster' 'sea_waves']


In [7]:
df_label = {0:'dog', 1:'chainsaw', 2:'crackling_fire', 3:'helicopter', 4:'rain',
            5:'crying_baby', 6:'clock_tick', 7:'sneezing', 8:'rooster', 9:'sea_waves'}

In [31]:
label2id = {v:k for k,v in df_label.items()}   # <-- 방향 뒤집기 중요!
df["y"] = df["category"].map(label2id)
assert df["y"].notna().all(), "라벨 매핑 실패: 철자 확인하세요."

In [36]:
def extract_features(file_path, sr=16000, n_mfcc=40):
    y, sr = librosa.load(file_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc, axis=1)

In [37]:
for idx, row in df.iterrows():
    wav_path = os.path.join(WAV_DIR, row["filename"])  # 파일 경로 만들기
    mfcc_vec = extract_mfcc(wav_path)                  # MFCC 벡터 추출
    X_features.append(mfcc_vec)                        # 특징 추가
    y_labels.append(row["df_label"])                          # 라벨 추가


In [38]:
X_features = np.array(X_features)
y_labels   = np.array(y_labels)

In [39]:
X = np.array(X)
y = np.array(y)

In [40]:
print("특징 배열 크기:", X_features.shape)  # (샘플 수, MFCC 개수)
print("라벨 배열 크기:", y_labels.shape)    # (샘플 수,)

특징 배열 크기: (400, 40)
라벨 배열 크기: (400,)


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, stratify=y_labels, random_state=42)

In [42]:
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [43]:
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
print(pred)
print(f"Test Accuracy: {acc:.3f}")

[2 5 3 4 8 2 2 7 4 0 2 2 5 2 4 0 8 0 9 0 0 9 8 7 7 1 1 6 3 8 0 3 9 5 1 9 5
 9 0 3 3 4 3 4 8 0 1 5 8 5 1 3 8 6 6 7 1 7 7 9 2 2 0 4 0 0 8 2 0 1 1 3 7 8
 3 4 0 2 8 9]
Test Accuracy: 0.738


In [46]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

CSV_PATH = 'C:\\Users\\ipl1\\Desktop\\IPL\\세미나(25하계)\\data\\sound\\esc50.csv'
WAV_DIR = 'C:\\Users\\ipl1\\Desktop\\IPL\\세미나(25하계)\\data\\sound\\audio\\audio'

# 1) CSV 로드 & ESC-10만 필터
df = pd.read_csv(CSV_PATH)
df = df[df["esc10"] == True].reset_index(drop=True)

label2id = {c:i for i,c in enumerate(sorted(df["category"].unique()))}
df["y"] = df["category"].map(label2id)

# 2) 특징 추출 함수 (MFCC 평균)
def extract_features(file_path, sr=16000, n_mfcc=40):
    y, sr = librosa.load(file_path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc, axis=1)  # (n_mfcc,)

# 3) 전체 데이터 처리
X, y = [], []
for _, row in df.iterrows():
    path = os.path.join(WAV_DIR, row["filename"])
    feat = extract_features(path)
    X.append(feat)
    y.append(row["y"])
X, y = np.array(X), np.array(y)

# 4) 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 5) RandomForest 학습
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

# 6) 평가
pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f"Test Accuracy: {acc:.3f}")


Test Accuracy: 0.775
